<a href="https://colab.research.google.com/github/VLFAri/Classificador-de-Imagens-de-Transporte/blob/main/Transportes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Importando o DataSet do Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive') #coloco o meu drive no sistema de arquivos do Google Colab

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/DataSet' #Caminho do diretório onde estão as imagens

# Importando biblioteca PyTorch e suas funcionalidades

In [ ]:
#IMporta a biblioteca PyTorch para modelos de aprendizado de máquina e deep learning
import torch

#Importa do PyTorch a classe ImagemFolder para organização do dataset de imagens em pastas
from torchvision.datasets import ImageFolder

#Importa do PyTorch o módulo transforms, usado para pré-processar imagens
from torchvision import transforms

#Classe DataLoader: cria um iterador sobre o dataset, dividindo-o em mini-batches, para depois ser utilizado no treinamento e validação
#Classe Subset: separa imagens para treino e validação
from torch.utils.data import DataLoader, Subset

#Importa a biblioteca NumPy, para cálculo de array e matrizes
import numpy as np

#Importa a função train_test_split do scikit-learn, para dividir dados em partes de treino e validação.
from sklearn.model_selection import train_test_split

#Importa o módulo nn (Neural Networks) do PyTorch, utilizada para utilização das camadas de rede neural
#Camadas utilizadas no módulo: nn.Linear e nn.Conv2D
import torch.nn as nn

#mporta o módulo functional, que contém as versões funcionais das operações das camadas, ex: ReLu
import torch.nn.functional as F

#Importa o módulo de otimizadores (nesse classificador o optimizador é o Adam)
import torch.optim as optim

In [ ]:
#Esses são os valores médios e desvios padrão dos canais RGB do teu conjunto de imagens.
#Utilizados para normalizar os dados
media = [0.4201163053512573, 0.4290950298309326, 0.4315476417541504]
desvio = [0.30978769063949585, 0.2988555133342743, 0.31542524695396423]

#Cria uma pipeline de transformações a ser aplicada em cada imagem do dataset.
#Resize: redimensiona todas as imagens para 224×224 pixels
#ToTensor: converte a imagem de formato PIL (ou NumPy) para tensor do PyTorch
#transforms.Normalize: normaliza cada canal RGB
t = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),transforms.Normalize(media, desvio)
])

#Carrega o dataset de imagens localizado em path através da classe ImageFOlder utilizando o pipeline de transformação criado antes
ds = ImageFolder(path, transform=t)

#Extrai os rótulos numéricos (classes) de todas as imagens e converte para um array NumPy.
#Facilita separação entre treinamento e validação
tg = np.array(ds.targets)

#Divide os índices do dataset em 70% treino e 30% validação.
#stratify=tg → garante que a proporção de classes (ex: gatos/cachorros) seja mantida em ambas as partes.
indice_treino, indice_validacao = train_test_split(np.arange(len(tg)),
               test_size=0.3, stratify=tg, random_state=42)

#Cria dois subconjuntos do dataset original (ds): um para treino e outro para validação
treino = Subset(ds, indice_treino)
validacao = Subset(ds, indice_validacao)

#Cria os DataLoaders, que controlam o envio de dados à GPU/CPU durante o treinamento.
#Batch de 64 imagens,
#shuffle=True → embaralha os dados de treino a cada época (evita que o modelo aprenda padrões errados na ordem).
#huffle=False → mantém a ordem na validação (não é necessário embaralhar).
#num_workers=2 → usa 2 threads para carregar as imagens em paralelo (acelera o processo de leitura do disco).
dl_treino = DataLoader(treino, batch_size = 64, shuffle=True, num_workers=2)
dl_validacao = DataLoader(validacao, batch_size = 64, shuffle=False, num_workers=2)

In [ ]:
#Cria uma classe que define o modelo de rede neural, herdando de nn.Module — a classe base de todos os modelos no PyTorch.
class ClassificarTipodeTransporte(nn.Module):
#O método __init__ é o construtor da classe, onde são definidas todas as camadas da rede.
  def __init__(self):
    #A linha super(...) chama o construtor da classe nn.Module para inicializar a estrutura básica do modelo.
    super(ClassificarTipodeTransporte, self).__init__()
    #c1, c2 e c3 -> camadas convolucionais
    #p1,p2 e p3 -> reduz a imagem pela metade em altura e largura, pegando o valor máximo em janelas 2×2.
    #fc1,fc2,fc3 -> camadas fully-connected (redes neurais tradicionais)
    self.c1 = nn.Conv2d(3,32,kernel_size=3,padding=1)
    self.p1 = nn.MaxPool2d(2,2)
    self.c2 = nn.Conv2d(32,64,kernel_size=3,padding=1)
    self.p2 = nn.MaxPool2d(2,2)
    self.c3 = nn.Conv2d(64,128,kernel_size=3,padding=1)
    self.p3 = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(128*28*28,512)
    self.fc2 = nn.Linear(512,128)
    self.fc3 = nn.Linear(128,5)

    #Adiciona dropout de 20%, uma técnica que desativa aleatoriamente 20% dos neurônios durante o treino
    #Evita overfitting (modelo memoriza demais e começa a decorar ao invés de aprender)
    self.dropout = nn.Dropout(0.2)

  #Define como os dados passam pelas camadas da rede (a arquitetura em si).
  def forward(self,x):
    #ReLU (função de ativação que zera valores negativos) -> introduz não-linearidade
    x = F.relu(self.c1(x))
    x = self.p1(x)
    x = F.relu(self.c2(x))
    x = self.p2(x)
    x = F.relu(self.c3(x))
    x = self.p3(x)

    #Achata (flatten) os dados: transforma o tensor 4D (batch, canais, altura, largura) em 2D (batch, total_de_valores_por_imagem)
    x = torch.flatten(x, 1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout(x)

    x = self.fc2(x)
    x = F.relu(x)
    x = self.dropout(x)

    x = self.fc3(x)

    return x #Retorna as previsões do modelo

In [ ]:
#Cria uma instância da tua classe de rede neural ClassificarTipodeTransporte
model = ClassificarTipodeTransporte()

#Verifica se há uma GPU (CUDA) disponível: se tiver, a usa e usa o processador caso contrário
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#Move todos os pesos e buffers do modelo para o dispositivo selecionado (GPU ou CPU).
model.to(device)

#Define a função de perda (loss function) usada para medir o erro do modelo.
criterion = nn.CrossEntropyLoss()

#Escolhe o otimizador Adam oara atualizar os pesos da rede com base nos gradientes calculados pela perda.
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#Pede ao usuário o número de épocas de treinamento.
#Cada época representa uma passagem completa por todo o conjunto de treino.
print("Quantidades de época que deseja treinar:",end=" ")
anos_treino = int(input())

#Criação de um loop externo, que vai repetir o treinamento para cada época.
for epoch in range(anos_treino):
  model.train() #Coloca o modelo em modo de treinamento.
  running_loss = 0.0 #Inicializa uma variável para acumular a perda (loss) de todos os lotes (batches) dessa época.
  for inputs, labels in dl_treino: #Loop interno: percorre o DataLoader de treino, que fornece os dados em batches

    #Move os dados do lote para o mesmo dispositivo do modelo (GPU ou CPU).
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad() #Zera os gradientes acumulados nas iterações anteriores (acumulados por padrão)
    outputs = model(inputs) #Envia o lote de imagens pelo modelo (executa o método forward)
    loss = criterion(outputs, labels) #Calcula a função de perda comparando as predições (outputs) com os rótulos verdadeiros (labels).
    loss.backward() #Daz o backpropagation (calcula os gradientes em relação à perda)
    optimizer.step() #O otimizador usa os gradientes calculados para atualizar os pesos da rede.

    #Adiciona o valor da perda (convertido para número comum) à variável running_loss, acumulando o erro de todos os batches
    running_loss += loss.item()

  #Ao final da época, exibe a perda média da época
  print(f"Loss da {epoch+1}ª época: {running_loss/len(dl_treino)}")

Quantidades de época que deseja treinar: 1
Loss da 1ª época: 1.613814937216895


In [ ]:
model.eval() #Coloca o modelo em modo de validação.

acerto = 0
total = 0
loss_total = 0

with torch.no_grad(): #Desativa o cálculo de gradientes

  #Cada lote (batch) de imagens e rótulos é passado pela rede.
  #outputs são as probabilidades ou pontuações de cada classe.
  for inputs, label in dl_validacao:
    inputs = inputs.to(device)
    label = label.to(device)
    outputs = model(inputs)

    #A função de custo (CrossEntropyLoss) mede o quanto o modelo está errando as previsões.
    #loss_total acumula o erro de todos os lotes.
    loss = criterion(outputs, label)
    loss_total += loss.item()

    #torch.max(outputs, 1) pega o índice da classe com maior pontuação (a previsão final)
    #Compara com os rótulos reais (label) e soma quantos estão corretos.
    #total é o total de amostras, e acerto é o total de acertos.
    _,predicao = torch.max(outputs, 1)
    total += label.size(0)
    acerto += (predicao == label).sum().item()

  loss_final = loss_total / len(dl_validacao) #erro médio por lote na validação.
  acuracia = (acerto/total)*100 #porcentagem de acertos do modelo.
  print(f"Acurácia: {acuracia}%")
  print(f"Loss: {loss_final}")


Acurácia: 35.6%
Loss: 1.523410141468048
